# Import Packages

In [ ]:
!pip install autokeras wandb

import numpy as np
import os
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import tensorflow as tf                
import pandas as pd
from tensorflow.keras.models import load_model
%matplotlib inline 

import os
import shutil

import autokeras as ak
import wandb
from wandb.keras import WandbCallback


In [ ]:
###################### Parameters ########################

project="Facial-keypoints"
run_id= "1.8"
resume_run = True

MAX_TRIALS=7
EPOCHS=10
PATIENCE=3

base_dir='/kaggle/input/facial-keypoints-detection/'
train_dir_zip=base_dir+'training.zip'
test_dir_zip=base_dir+'test.zip'
##########################################################



In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
key = user_secrets.get_secret("wandb_auth")

wandb.login(key=key)
wandb.init(project=project,id=run_id, resume=resume_run)

# Loading the Data
We have to write a load_data function that load the images and the labels from the folder.

In [ ]:
from zipfile import ZipFile
with ZipFile(train_dir_zip,'r') as zipObj:
    zipObj.extractall('.')
    print("Train Archive unzipped")
with ZipFile(test_dir_zip,'r') as zipObj:
    zipObj.extractall('.')
    print("Test Archive unzipped")

In [ ]:
train_dir='./training.csv'
test_dir='./test.csv'
train=pd.read_csv(train_dir)
test=pd.read_csv(test_dir)

In [ ]:
train=train.dropna()
train=train.reset_index(drop=True)

In [ ]:
X_train=[]
Y_train=[]

for img in train['Image']:
    X_train.append(np.asarray(img.split(),dtype=float).reshape(96,96,1))
X_train=np.reshape(X_train,(-1,96,96,1))
X_train = np.asarray(X_train).astype('float32')
    
for i in range(len((train))): 
    Y_train.append(np.asarray(train.iloc[i][0:30].to_numpy()))
Y_train = np.asarray(Y_train).astype('float32')

In [ ]:
X_test=[]
for img in test['Image']:
    X_test.append(np.asarray(img.split(),dtype=float).reshape(96,96,1))
X_test=np.reshape(X_test,(-1,96,96,1))
X_test = np.asarray(X_test).astype('float32')

In [ ]:
disp=8

fig,axes=plt.subplots((disp+3)//4,4,figsize=(15,10))
                    
for i in range(disp):
    axes[i//4,i%4].imshow(X_train[i].reshape(96,96),cmap='gray')
    axes[i//4,i%4].scatter([train[train.columns[2*j]][i] for j in range(15)],[train[train.columns[2*j+1]][i] for j in range(15)],s=10,c='r')

## Train the model

We will train an EfficientNet model using autokeras library 

In [ ]:
      
CALLBACKS = [
    tf.keras.callbacks.EarlyStopping(patience=PATIENCE),
    WandbCallback(),
    #CustomCallback(),
]

In [ ]:
reg = ak.ImageRegressor(max_trials=MAX_TRIALS)

#input_node = ak.ImageInput()
#output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
#output_node = ak.ResNetBlock(version="v2")(output_node)
#output_node = ak.RegressionHead()(output_node)
#reg = ak.AutoModel(
#    inputs=input_node, outputs=output_node, max_trials=MAX_TRIALS
#)

# Feed the image classifier with training data.                                                   
reg.fit(X_train, Y_train, validation_split=0.15, epochs=EPOCHS, callbacks=CALLBACKS)                                                     

## Evaluate the model

We will evaluate the model on the test set

In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
disp=8

fig,axes=plt.subplots((disp+3)//4,4,figsize=(15,10))
                    
for i in range(disp):
    axes[i//4,i%4].imshow(X_test[i].reshape(96,96),cmap='gray')
    axes[i//4,i%4].scatter([y_pred[i][2*j] for j in range(15)],[y_pred[i][2*j+1] for j in range(15)],s=10,c='r')

In [ ]:
y_pred= y_pred.reshape(-1,)

In [ ]:
y_pred.shape

In [ ]:
#create submission csv file
submission = pd.DataFrame({'Location': y_pred})
submission.to_csv('submission.csv', index=True , index_label='RowId')